In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torch.nn.functional as F
from transformers import AdamW
import pandas as pd
from transformers import BertTokenizer, BertModel, BertConfig
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import csv
import re
import validators
import emoji
import unidecode
import nltk
import pickle
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Mai\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
# Set the seed for reproducibility
SEED = 1235
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

# BERT Hyperparameters (ADDITION)
n_bert_layers = 16  # Assuming the base model has 12 layers
bert_lr = 0.001
pooling_strategy = 'cls'  # Options: 'cls', 'mean', 'max'
bert_hidden_size = 768  # Adjust based on your BERT model
max_seq_length = 128
fine_tune_strategy = 'full'  # Options: 'full', 'last_layer'
bert_dropout = 0.9  # Adjust based on BERT model specifications

max_seq_length = 128  # This should match the max_seq_length used in BERT model
padding_strategy = 'max_length'  # Options: 'max_length', 'do_not_pad', 'longest'
truncation_strategy = 'longest_first'  # Options: 'longest_first', 'only_first', 'only_second'
do_lower_case = True  # Set to False if using a cased model

config = BertConfig(
    num_hidden_layers=n_bert_layers,
    hidden_size=bert_hidden_size,
    num_attention_heads=24,  # Assuming 12 attention heads
    intermediate_size=4 * bert_hidden_size,  # Default value in BERT
    hidden_dropout_prob=bert_dropout,
    attention_probs_dropout_prob=bert_dropout,
)

# Load BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
                                          max_length=max_seq_length,
                                          padding=padding_strategy,
                                          truncation=truncation_strategy,
                                          do_lower_case=do_lower_case)
# Load the BERT model with the custom configuration
bert_model = BertModel(config=config)


In [3]:
data_path = 'C:/Users/Mai/thesis/ch 4/dataset.csv'
data_df = pd.read_csv(data_path)
data_df = data_df.rename(columns={'Tweet Content': 'text', 'Label': 'label'})

In [4]:
data_df.head()

,text,Sentiment,label
0,Worst Bong ever. https://t.co/QA7R8VYppC,Negative,Hate
1,what i dont like about leni robredo's platform...,Negative,Hate
2,Ito ang tunay na survey ni VP Leni Robredo #1 ...,Negative,Hate
3,(3) BBM sued for Pork Barrel Scam\n\nhttps://t...,Negative,Hate
4,Sabog din sumagot tong si Norberto Gonzales no...,Negative,Hate


In [5]:
groupedby_sentiment = data_df.groupby(data_df.Sentiment)
data_df_positive = groupedby_sentiment.get_group("Positive")
data_df_positive

,text,Sentiment,label
2560,I took The Blind Test and my top candidates ar...,Positive,Non-hate
2561,"""True leader show up and man up."" - VP Leni Ro...",Positive,Non-hate
2562,Leni Robredo for president cutie 🤞🌸,Positive,Non-hate
2563,"Ako si Christian Tan, kabataan at kaisa ni Bon...",Positive,Non-hate
2564,Ate @xlykable Let’s support VP Leni and Sen. K...,Positive,Non-hate
...,...,...,...
3835,Just because Aiai did not supported Leni Robre...,Positive,Non-hate
3836,"“Mga kababayan, summon the warrior in you and ...",Positive,Non-hate
3837,@thekiarasworld Now I know that not all of the...,Positive,Non-hate
3838,Ping Lacson Ang may Plano sa bansa\n\n#KayPing...,Positive,Non-hate


In [6]:
data_df_negative = groupedby_sentiment.get_group("Negative")
data_df_negative

,text,Sentiment,label
0,Worst Bong ever. https://t.co/QA7R8VYppC,Negative,Hate
1,what i dont like about leni robredo's platform...,Negative,Hate
2,Ito ang tunay na survey ni VP Leni Robredo #1 ...,Negative,Hate
3,(3) BBM sued for Pork Barrel Scam\n\nhttps://t...,Negative,Hate
4,Sabog din sumagot tong si Norberto Gonzales no...,Negative,Hate
...,...,...,...
2555,Headline: The ambitious presidential candidate...,Negative,Hate
2556,"Norberto Gonzales is right, its a missed oppor...",Negative,Hate
2557,"The audacity to call Leni Robredo ""bobo"", "" ta...",Negative,Hate
2558,Bongbong Marcos is a Nazi. https://t.co/gY3xHb...,Negative,Hate


In [7]:
data_df_neutral = groupedby_sentiment.get_group("Neutral")
data_df_neutral

,text,Sentiment,label
3840,bongbong marcos dot com,Neutral,Non-hate
3841,Grabe pala talaga yung actions ni Leni Robredo...,Neutral,Non-hate
3842,"“Ngayong darating na halalan, ang tatanglaw sa...",Neutral,Non-hate
3843,For this COMELEC debate:\n\nValedictorian: Len...,Neutral,Non-hate
3844,Focus on the ball kakampinks\n\nPresident Leni...,Neutral,Non-hate
...,...,...,...
5115,President Leni Robredo and Vice President Kiko...,Neutral,Non-hate
5116,@jillrobredo 🌺🌺🌺\nthank you din kay @maraceped...,Neutral,Non-hate
5117,LOOK: Presidential candidate Bongbong Marcos m...,Neutral,Non-hate
5118,@itsmaxandcheese Leni Robredo for President 2022,Neutral,Non-hate


In [8]:
#binary hate non-hate
data_df_hate = data_df_negative.sample(n = 2560, replace=True)

data_df_positive = data_df_positive.sample(n = 1280, replace=True)
data_df_neutral = data_df_neutral.sample(n = 1280, replace=True)

data_df_nonhate = data_df_positive.append(data_df_neutral)

data_df = data_df_hate.append(data_df_nonhate)

C:\Users\Mai\AppData\Local\Temp\ipykernel_14080\2699631638.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_df_nonhate = data_df_positive.append(data_df_neutral)
C:\Users\Mai\AppData\Local\Temp\ipykernel_14080\2699631638.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_df = data_df_hate.append(data_df_nonhate)


In [9]:
data_df = data_df.drop(['Sentiment'], axis = 1)

In [10]:
#data_df.to_csv('dataset.csv', index=False)
data_df

,text,label
1942,@MamaHopia KAHIT ANONG GAWIN MONG PAGPAPALIT N...,Hate
1441,pano ba yan eh mismong si ping lacson isang re...,Hate
567,Magiging Author na si Bongbong Marcos. Magsusu...,Hate
1620,@ToniSpeakEasy KAHIT MAGLUPASAY PA SIYA LENI R...,Hate
2368,"@UPThirdWorld Studies Center: ""His father, the...",Hate
...,...,...
4350,One of Leni Robredo's Senatorial bets! Napa-se...,Non-hate
3866,"Ka Leody De Guzman, Sen. Ping Lacson, and VP L...",Non-hate
4507,"Miss Leni Robredo &amp; Mr Kiko Pangilinan, al...",Non-hate
4870,"MULA SA IBABA, HINDI SA TUKTOK'\n\nPresidentia...",Non-hate


In [11]:
data_df = data_df.reset_index()

In [12]:
data_df

,index,text,label
0,1942,@MamaHopia KAHIT ANONG GAWIN MONG PAGPAPALIT N...,Hate
1,1441,pano ba yan eh mismong si ping lacson isang re...,Hate
2,567,Magiging Author na si Bongbong Marcos. Magsusu...,Hate
3,1620,@ToniSpeakEasy KAHIT MAGLUPASAY PA SIYA LENI R...,Hate
4,2368,"@UPThirdWorld Studies Center: ""His father, the...",Hate
...,...,...,...
5115,4350,One of Leni Robredo's Senatorial bets! Napa-se...,Non-hate
5116,3866,"Ka Leody De Guzman, Sen. Ping Lacson, and VP L...",Non-hate
5117,4507,"Miss Leni Robredo &amp; Mr Kiko Pangilinan, al...",Non-hate
5118,4870,"MULA SA IBABA, HINDI SA TUKTOK'\n\nPresidentia...",Non-hate


In [13]:
# data_df = "dataset.csv"

In [14]:
# Preprocessing function
def preprocess_text(text):
    tokens = tokenizer.tokenize(text)
    tokens = tokens[:tokenizer.model_max_length - 2]  # Account for [CLS] and [SEP] tokens
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokens)
    return indexed_tokens

filipino_stopwords = set(
    """
akin
aking
ako
alin
am
amin
aming
ang
ano
anumang
apat
at
atin
ating
ay
bababa
bago
bakit
bawat
bilang
dahil
dalawa
dapat
din
dito
doon
gagawin
gayunman
ginagawa
ginawa
ginawang
gumawa
gusto
habang
hanggang
hindi
huwag
iba
ibaba
ibabaw
ibig
ikaw
ilagay
ilalim
ilan
inyong
isa
isang
itaas
ito
iyo
iyon
iyong
ka
kahit
kailangan
kailanman
kami
kanila
kanilang
kanino
kanya
kanyang
kapag
kapwa
karamihan
katiyakan
katulad
kaya
kaysa
ko
kong
kulang
kumuha
kung
laban
lahat
lamang
likod
lima
maaari
maaaring
maging
mahusay
makita
marami
marapat
masyado
may
mayroon
mga
minsan
mismo
mula
muli
na
nabanggit
naging
nagkaroon
nais
nakita
namin
napaka
narito
nasaan
ng
ngayon
ni
nila
nilang
nito
niya
niyang
noon
o
pa
paano
pababa
paggawa
pagitan
pagkakaroon
pagkatapos
palabas
pamamagitan
panahon
pangalawa
para
paraan
pareho
pataas
pero
pumunta
pumupunta
sa
saan
sabi
sabihin
sarili
sila
sino
siya
tatlo
tayo
tulad
tungkol
una
walang
""".split()
)

# Date De-Identification
def remove_mentions(text):
    mention_pattern = re.compile(r'@\w+')
    
    # Use re.sub to remove mentions
    cleaned_text = mention_pattern.sub('', text)
    
    # Remove extra spaces and strip leading/trailing spaces
    cleaned_text = ' '.join(cleaned_text.split())

    return cleaned_text

# URL Removal
def remove_url(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    
    # Use re.sub to remove URLs
    cleaned_text = url_pattern.sub('', text)
    
    # Remove extra spaces and strip leading/trailing spaces
    cleaned_text = ' '.join(cleaned_text.split())

    return cleaned_text

# Special Characters Removal
def remove_special_characters(text):
    text = emoji.replace_emoji(text, replace="[emoji]")
    
    # Split the text into words
    words = text.split(" ")
    
    # Initialize an empty string to store the cleaned text
    cleaned_text = ""
    
    # Iterate through each word
    for word in words:
        # Check if the word contains only special characters or "[emoji]"
        if not (re.match(r"^[_\W]+$", word) or "[emoji]" in word):
            if len(cleaned_text) == 0:
                cleaned_text = f"{word}"
            else:
                cleaned_text = f"{cleaned_text} {word}"
                
    # Remove diacritics
    text_no_diacritics = unidecode.unidecode(cleaned_text)

    # Split the text into words
    sentence = text_no_diacritics.split(" ")
    output = ""

    # Remove special characters and numerics
    for part in sentence:
        part = re.sub("[^A-Za-z ]+$", "", part)
        part = re.sub("^[^A-Za-z #]+", "", part)
        if not (len(part) <= 1 or re.match(r"[^a-zA-Z#]", part)):
            if len(output) == 0:
                output = f"{part}"
            else:
                output = f"{output} {part}"

    # Remove extra spaces and strip leading/trailing spaces
    cleaned_text = ' '.join(output.split())

    return cleaned_text

# Remove English Stop Words
from nltk.corpus import stopwords
english_stopwords = stopwords.words('english')

# Remove English Stop Words
def remove_english_stopwords(text):
    stop_words = set(stopwords.words('english'))
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    
    # Remove extra spaces and strip leading/trailing spaces
    cleaned_text = ' '.join(filtered_words)
    
    return cleaned_text

# Remove Filipino Stop Words
def remove_filipino_stopwords(text):
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in filipino_stopwords]
    
    # Remove extra spaces and strip leading/trailing spaces
    cleaned_text = ' '.join(filtered_words)
  
    return cleaned_text

# Candidate Name Removal
def remove_candidate_names(text):
    candidatelist = "leni robredo bongbong marcos isko moreno domagoso manny pacman pacquiao ping lacson ernie abella leody de guzman norberto gonzales jose montemayor jr faisal mangondato"
    candidatelist = candidatelist.split()
    candidate_pattern = re.compile(r'\b(?:' + '|'.join(map(re.escape, candidatelist)) + r')\b', re.IGNORECASE)
    
    # Use re.sub to remove candidate names
    cleaned_text = candidate_pattern.sub('', text)
    
    # Remove extra spaces and strip leading/trailing spaces
    cleaned_text = ' '.join(cleaned_text.split())
        
    return cleaned_text

# Hashtag Removal
def remove_hashtags(text):
    # Split the text into words
    words = text.split()
    
    # Initialize an empty list to store cleaned words
    cleaned_words = []
    
    for word in words:
        # Check if the word is a hashtag (starts with #)
        if not word.startswith('#'):
            cleaned_words.append(word)
    
    # Join the cleaned words into a single string
    cleaned_text = ' '.join(cleaned_words)
    
    return cleaned_text

In [15]:
#PreProcessing

In [16]:
# Data De-Identification
data_df['text'] = data_df['text'].apply(remove_mentions)

for i in range(10):
    text = data_df["text"][i]
    label = data_df["label"][i]

    print('Text: ', text, "\n\nLabel: ", label, "\n\n~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~\n")

Text:  KAHIT ANONG GAWIN MONG PAGPAPALIT NG KULAY LENI ROBREDO,HINDI MO NA MAALIS O MA-ERASE SA INYONG KATAUHAN ANG PAGIGING DILAWAN,TATAK DILAWAN KA TALAGA,YUN YONG NAKATATAK NA RIN SA AMING ISIPAN BILANG MGA MAMAYAN AT MGA VOTERS. 

Label:  Hate 

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Text:  pano ba yan eh mismong si ping lacson isang red tagger 😞 #PiliPinasDebates2022 

Label:  Hate 

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Text:  Magiging Author na si Bongbong Marcos. Magsusulat na sya ng SALN, Accounting Equation nya Debit minus Credit Equals KUPIT Anong Accounting book kaya ito? #ManggagawaVSMagnanakaw https://t.co/0aty2rZcSz 

Label:  Hate 

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Text:  KAHIT MAGLUPASAY PA SIYA LENI ROBREDO KIKO PANGILINAN PA RIN AKO 

Label:  Hate 

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Text:  Studies Center: "His father, the dictator, lied about his son’s degree. His mother... flaunted the same lie. And the son,

In [17]:
# URL Removal
data_df['text'] = data_df['text'].apply(remove_url)

for i in range(10):
    text = data_df["text"][i]
    label = data_df["label"][i]

    print('Text: ', text, "\n\nLabel: ", label, "\n\n~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~\n")

Text:  KAHIT ANONG GAWIN MONG PAGPAPALIT NG KULAY LENI ROBREDO,HINDI MO NA MAALIS O MA-ERASE SA INYONG KATAUHAN ANG PAGIGING DILAWAN,TATAK DILAWAN KA TALAGA,YUN YONG NAKATATAK NA RIN SA AMING ISIPAN BILANG MGA MAMAYAN AT MGA VOTERS. 

Label:  Hate 

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Text:  pano ba yan eh mismong si ping lacson isang red tagger 😞 #PiliPinasDebates2022 

Label:  Hate 

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Text:  Magiging Author na si Bongbong Marcos. Magsusulat na sya ng SALN, Accounting Equation nya Debit minus Credit Equals KUPIT Anong Accounting book kaya ito? #ManggagawaVSMagnanakaw 

Label:  Hate 

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Text:  KAHIT MAGLUPASAY PA SIYA LENI ROBREDO KIKO PANGILINAN PA RIN AKO 

Label:  Hate 

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Text:  Studies Center: "His father, the dictator, lied about his son’s degree. His mother... flaunted the same lie. And the son, who now wants to be pre

In [18]:
# Special Characters Removal
data_df['text'] = data_df['text'].apply(remove_special_characters)

for i in range(10):
    text = data_df["text"][i]
    label = data_df["label"][i]

    print('Text: ', text, "\n\nLabel: ", label, "\n\n~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~\n")

Text:  KAHIT ANONG GAWIN MONG PAGPAPALIT NG KULAY LENI ROBREDO,HINDI MO NA MAALIS MA-ERASE SA INYONG KATAUHAN ANG PAGIGING DILAWAN,TATAK DILAWAN KA TALAGA,YUN YONG NAKATATAK NA RIN SA AMING ISIPAN BILANG MGA MAMAYAN AT MGA VOTERS 

Label:  Hate 

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Text:  pano ba yan eh mismong si ping lacson isang red tagger #PiliPinasDebates 

Label:  Hate 

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Text:  Magiging Author na si Bongbong Marcos Magsusulat na sya ng SALN Accounting Equation nya Debit minus Credit Equals KUPIT Anong Accounting book kaya ito #ManggagawaVSMagnanakaw 

Label:  Hate 

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Text:  KAHIT MAGLUPASAY PA SIYA LENI ROBREDO KIKO PANGILINAN PA RIN AKO 

Label:  Hate 

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Text:  Studies Center His father the dictator lied about his son's degree His mother flaunted the same lie And the son who now wants to be president continues the l

In [19]:
# Lowercase
data_df['text'] = data_df['text'].str.lower()

for i in range(10):
    text = data_df["text"][i]
    label = data_df["label"][i]

    print('Text: ', text, "\n\nLabel: ", label, "\n\n~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~\n")

Text:  kahit anong gawin mong pagpapalit ng kulay leni robredo,hindi mo na maalis ma-erase sa inyong katauhan ang pagiging dilawan,tatak dilawan ka talaga,yun yong nakatatak na rin sa aming isipan bilang mga mamayan at mga voters 

Label:  Hate 

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Text:  pano ba yan eh mismong si ping lacson isang red tagger #pilipinasdebates 

Label:  Hate 

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Text:  magiging author na si bongbong marcos magsusulat na sya ng saln accounting equation nya debit minus credit equals kupit anong accounting book kaya ito #manggagawavsmagnanakaw 

Label:  Hate 

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Text:  kahit maglupasay pa siya leni robredo kiko pangilinan pa rin ako 

Label:  Hate 

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Text:  studies center his father the dictator lied about his son's degree his mother flaunted the same lie and the son who now wants to be president continues the l

In [20]:
# Remove English Stop Words
# data_df['text'] = data_df['text'].apply(remove_english_stopwords)

# for i in range(10):
#    text = data_df["text"][i]
#    label = data_df["label"][i]

#    print('Text: ', text, "\n\nLabel: ", label, "\n\n~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~\n")

In [21]:
# Remove Filipino Stop Words
# data_df['text'] = data_df['text'].apply(remove_filipino_stopwords)

# for i in range(10):
#    text = data_df["text"][i]
#    label = data_df["label"][i]

#    print('Text: ', text, "\n\nLabel: ", label, "\n\n~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~\n")

In [22]:
# Remove Candidate Names
# data_df['text'] = data_df['text'].apply(remove_candidate_names)

# for i in range(10):
    # text = data_df["text"][i]
    # label = data_df["label"][i]

    # print('Text: ', text, "\n\nLabel: ", label, "\n\n~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~\n")

In [23]:
# Remove Hashtags
data_df['text'] = data_df['text'].apply(remove_hashtags)

for i in range(10):
    text = data_df["text"][i]
    label = data_df["label"][i]

    print('Text: ', text, "\n\nLabel: ", label, "\n\n~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~\n")

Text:  kahit anong gawin mong pagpapalit ng kulay leni robredo,hindi mo na maalis ma-erase sa inyong katauhan ang pagiging dilawan,tatak dilawan ka talaga,yun yong nakatatak na rin sa aming isipan bilang mga mamayan at mga voters 

Label:  Hate 

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Text:  pano ba yan eh mismong si ping lacson isang red tagger 

Label:  Hate 

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Text:  magiging author na si bongbong marcos magsusulat na sya ng saln accounting equation nya debit minus credit equals kupit anong accounting book kaya ito 

Label:  Hate 

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Text:  kahit maglupasay pa siya leni robredo kiko pangilinan pa rin ako 

Label:  Hate 

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Text:  studies center his father the dictator lied about his son's degree his mother flaunted the same lie and the son who now wants to be president continues the lie for the past years the filipino people 

In [24]:
data_df.to_csv('binary8.csv', index=False)

In [25]:
data_df

,index,text,label
0,1942,kahit anong gawin mong pagpapalit ng kulay len...,Hate
1,1441,pano ba yan eh mismong si ping lacson isang re...,Hate
2,567,magiging author na si bongbong marcos magsusul...,Hate
3,1620,kahit maglupasay pa siya leni robredo kiko pan...,Hate
4,2368,studies center his father the dictator lied ab...,Hate
...,...,...,...
5115,4350,one of leni robredo's senatorial bets napa-sea...,Non-hate
5116,3866,ka leody de guzman sen ping lacson and vp leni...,Non-hate
5117,4507,miss leni robredo amp mr kiko pangilinan alway...,Non-hate
5118,4870,mula sa ibaba hindi sa tuktok presidential asp...,Non-hate


In [26]:
data_df['text'] = data_df['text'].apply(preprocess_text)

In [27]:
data_df

,index,text,label
0,1942,"[10556, 16584, 2019, 5063, 11721, 10105, 12256...",Hate
1,1441,"[6090, 2080, 8670, 13619, 15501, 28616, 8202, ...",Hate
2,567,"[23848, 8004, 2075, 3166, 6583, 9033, 14753, 1...",Hate
3,1620,"[10556, 16584, 23848, 7630, 19707, 4710, 6643,...",Hate
4,2368,"[2913, 2415, 2010, 2269, 1996, 21237, 9828, 20...",Hate
...,...,...,...
5115,4350,"[2028, 1997, 18798, 2072, 6487, 23417, 1005, 1...",Non-hate
5116,3866,"[10556, 6688, 5149, 2139, 22789, 12411, 17852,...",Non-hate
5117,4507,"[3335, 18798, 2072, 6487, 23417, 23713, 2720, ...",Non-hate
5118,4870,"[14163, 2721, 7842, 21307, 19736, 9269, 7842, ...",Non-hate


In [28]:
train_df, test_df = train_test_split(data_df, test_size=0.3, random_state=SEED)

In [29]:
# Define the CNN model
class CNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, dropout):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.embedding_dropout = nn.Dropout(dropout) 
        self.conv1 = nn.Conv1d(in_channels=embedding_dim, out_channels=256, kernel_size=2, padding='same')
        self.pool1 = nn.MaxPool1d(2, 2)
        self.conv2 = nn.Conv1d(in_channels=256, out_channels=64, kernel_size=4, padding='same')
        self.global_pooling = nn.AdaptiveMaxPool1d(1)
        self.dropout = nn.Dropout(0.5)
        self.fc = nn.Linear(64, output_dim)

    def forward(self, x):
        embedded = self.embedding(x)
        embedded = self.embedding_dropout(embedded)
        x = embedded.permute(0, 2, 1)  # Change the dimensions for convolution
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        x = F.relu(self.conv2(x))
        x = self.global_pooling(x).squeeze(2)
        x = self.dropout(x)
        x = self.fc(x)
        return x

In [30]:
# Set up iterators
BATCH_SIZE = 64

In [31]:
class TextDataset(torch.utils.data.Dataset):
    def __init__(self, dataframe, max_seq_length):
        self.data = dataframe
        self.max_seq_length = max_seq_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text = self.data.iloc[idx]['text']
        label = self.data.iloc[idx]['label']

        # Padding and conversion to tensor
        padded_text = torch.tensor(text[:self.max_seq_length] + [0] * (self.max_seq_length - len(text)))
        return padded_text, label

In [32]:
train_dataset = TextDataset(train_df, 1000)
test_dataset = TextDataset(test_df, 1000)

In [33]:
train_iterator = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_iterator = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [34]:
# Define model hyperparameters
VOCAB_SIZE = tokenizer.vocab_size
EMBEDDING_DIM = 768
N_FILTERS = 100
FILTER_SIZES = [3, 4, 5]
OUTPUT_DIM = 1
DROPOUT = 0.5

# CNN Hyperparameters
hidden_dim = 100
n_conv_layers = 1
kernel_sizes = [2, 3, 4]
activation = nn.ReLU()

In [35]:
# Initialize CNN model
model = CNN(VOCAB_SIZE, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT)
#Initialize CNN model
# model = CNN(VOCAB_SIZE, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT, hidden_dim, n_conv_layers, kernel_sizes, activation)

In [36]:
# Initialize BERT model (for embedding extraction)
bert_model.eval()  # Set to evaluation mode

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.9, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-15): 16 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.9, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.9, inplace=False)
  

In [37]:
with torch.no_grad():
    for i, token in enumerate(tokenizer.get_vocab()):
        token_id = tokenizer.convert_tokens_to_ids(token)
        token_embedding = bert_model.embeddings.word_embeddings.weight[token_id]
        model.embedding.weight[i].data.copy_(token_embedding)

bert_parameters = []
for layer in bert_model.encoder.layer:
    bert_parameters.extend(layer.parameters())

# Create AdamW optimizer with custom hyperparameters for BERT embeddings
bert_learning_rate = 2e-4  # Adjust as needed
bert_optimizer = optim.AdamW(bert_parameters, lr=bert_learning_rate)

In [38]:
data_df

,index,text,label
0,1942,"[10556, 16584, 2019, 5063, 11721, 10105, 12256...",Hate
1,1441,"[6090, 2080, 8670, 13619, 15501, 28616, 8202, ...",Hate
2,567,"[23848, 8004, 2075, 3166, 6583, 9033, 14753, 1...",Hate
3,1620,"[10556, 16584, 23848, 7630, 19707, 4710, 6643,...",Hate
4,2368,"[2913, 2415, 2010, 2269, 1996, 21237, 9828, 20...",Hate
...,...,...,...
5115,4350,"[2028, 1997, 18798, 2072, 6487, 23417, 1005, 1...",Non-hate
5116,3866,"[10556, 6688, 5149, 2139, 22789, 12411, 17852,...",Non-hate
5117,4507,"[3335, 18798, 2072, 6487, 23417, 23713, 2720, ...",Non-hate
5118,4870,"[14163, 2721, 7842, 21307, 19736, 9269, 7842, ...",Non-hate


In [39]:
#optimizer = optim.Adam(model.parameters())
# Your custom hyperparameters
learning_rate = 0.001
beta_1 = 0.9
beta_2 = 0.999
epsilon = 1e-08
weight_decay = 0.0

# Create Adam optimizer with custom hyperparameters
optimizer = optim.Adam(model.parameters(), lr=learning_rate, betas=(beta_1, beta_2), eps=epsilon, weight_decay=weight_decay)
criterion = nn.BCEWithLogitsLoss()

In [40]:
# Train function
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def train(model, iterator):
    model.train()
    epoch_loss = 0
    for text_batch, label_batch in iterator:
        # Extract text sequences from the text_batch tensor
        texts = text_batch
        
        # Extract and process labels
        labels = [1 if label == 'Hate' else 0 for label in label_batch]  # Example conversion
        
        texts = texts.to(device)  # Move to device if needed
        labels = torch.tensor(labels, dtype=torch.float32).to(device)  # Convert to tensor
        
        optimizer.zero_grad()
        predictions = model(texts).squeeze(1)
        
        loss = criterion(predictions, labels)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    return epoch_loss / len(iterator)


# Evaluate function
def evaluate(model, iterator):
    model.eval()
    epoch_loss = 0
    predicted_labels = []
    true_labels = []  # Declare the true_labels list
    
    with torch.no_grad():
        for text_batch, label_batch in iterator:
            texts = text_batch  # Extract text sequences
            labels = [1 if label == 'Hate' else 0 for label in label_batch]  # Example conversion
            
            texts = texts.to(device)  # Move to device
            labels = torch.tensor(labels, dtype=torch.float32).to(device)  # Convert to tensor
            
            predictions = model(texts).squeeze(1)
            loss = criterion(predictions, labels)
            
            epoch_loss += loss.item()
            predicted_labels.extend(torch.round(torch.sigmoid(predictions)).cpu().numpy())
            true_labels.extend(labels.cpu().numpy())
    
    # Calculate accuracy, f1, precision, recall
    accuracy = accuracy_score(true_labels, predicted_labels)
    f1 = f1_score(true_labels, predicted_labels)
    precision = precision_score(true_labels, predicted_labels)
    recall = recall_score(true_labels, predicted_labels)
    
    return epoch_loss / len(iterator), accuracy, f1, precision, recall

In [41]:
N_EPOCHS = 10
for epoch in range(N_EPOCHS):
    train_loss = train(model, train_iterator)
    test_loss, accuracy, f1, precision, recall = evaluate(model, test_iterator)

    print(f'Epoch: {epoch+1:02}')
    print(f'\tTrain Loss: {train_loss:.3f}')
    print(f'\tTest Loss: {test_loss:.3f}')
    print(f'\tAccuracy: {accuracy:.4f} | F1-Score: {f1:.4f}')
    print(f'\tPrecision: {precision:.4f} | Recall: {recall:.4f}')

D:\Users\Mai\anaconda3\Lib\site-packages\torch\nn\modules\conv.py:309: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at C:\b\abs_abjetg6_iu\croot\pytorch_1686932924616\work\aten\src\ATen\native\Convolution.cpp:1004.)
  return F.conv1d(input, weight, bias, self.stride,


Epoch: 01
	Train Loss: 0.601
	Test Loss: 0.440
	Accuracy: 0.8047 | F1-Score: 0.8246
	Precision: 0.7382 | Recall: 0.9338
Epoch: 02
	Train Loss: 0.325
	Test Loss: 0.315
	Accuracy: 0.8763 | F1-Score: 0.8773
	Precision: 0.8562 | Recall: 0.8993
Epoch: 03
	Train Loss: 0.180
	Test Loss: 0.308
	Accuracy: 0.8893 | F1-Score: 0.8892
	Precision: 0.8755 | Recall: 0.9033
Epoch: 04
	Train Loss: 0.105
	Test Loss: 0.318
	Accuracy: 0.8861 | F1-Score: 0.8854
	Precision: 0.8756 | Recall: 0.8954
Epoch: 05
	Train Loss: 0.063
	Test Loss: 0.360
	Accuracy: 0.8997 | F1-Score: 0.8979
	Precision: 0.8991 | Recall: 0.8967
Epoch: 06
	Train Loss: 0.043
	Test Loss: 0.394
	Accuracy: 0.8984 | F1-Score: 0.8976
	Precision: 0.8895 | Recall: 0.9060
Epoch: 07
	Train Loss: 0.032
	Test Loss: 0.403
	Accuracy: 0.8978 | F1-Score: 0.8962
	Precision: 0.8945 | Recall: 0.8980
Epoch: 08
	Train Loss: 0.027
	Test Loss: 0.425
	Accuracy: 0.8991 | F1-Score: 0.8971
	Precision: 0.8989 | Recall: 0.8954
Epoch: 09
	Train Loss: 0.023
	Test Loss: